In [ ]:
# LO CIRCUITS
# can always get a unitary matrix from a LO circuit in Perceval
# we can convert a matrix into a ZW circuit in optyx
# the opposite way is how we evaluate LO circuits in perceval

In [ ]:
# MORE GENERAL CIRCUITS
#

In [1]:
import numpy as np

import perceval as pcvl
from perceval import catalog

In [2]:
p = pcvl.Processor("SLOS", 6)
p.add(0, catalog["postprocessed cnot"].build_processor())
p.add(0, pcvl.BS.H());

In [4]:
# 2 measured modes
# offset = 0 means that there is 0 empty modes between the measured modes and the circuit
# the default circuit is an empty circuit
ff_X = pcvl.FFCircuitProvider(2, 0, pcvl.Circuit(2))

# Now if we measure a logical state |1>, we need to perform a permutation of the modes
ff_X.add_configuration([0, 1], pcvl.PERM([1, 0]))

# Add perfect detectors to the modes that will be measured
p.add(2, pcvl.Detector.pnr())
p.add(3, pcvl.Detector.pnr())
p.add(2, ff_X);

In [5]:
phi = pcvl.P("phi")
# Like Circuits and Processors, we can chain the `add` methods
ff_Z = pcvl.FFConfigurator(2, 3, pcvl.PS(phi), {"phi": 0}).add_configuration([0, 1], {"phi": np.pi})

p.add(0, pcvl.Detector.pnr())
p.add(1, pcvl.Detector.pnr())
p.add(0, ff_Z);

In [8]:
p.components

[[[4, 5, 6, 7],
 [[0, 1, 2, 3, 4, 5],
 [[4, 5, 6, 7],
 ((0, 1), <perceval.components.unitary_components.BS at 0x71aded2ca9e0>),
 ((0, 1, 2, 3, 4, 5),
  <perceval.components.unitary_components.Barrier at 0x71aded105ae0>),
 ((0, 1), <perceval.components.unitary_components.Barrier at 0x71aded106d70>),
 ((2,), <perceval.components.detector.Detector at 0x71adf1766140>),
 ((3,), <perceval.components.detector.Detector at 0x71aded2ca5c0>),
 ((4, 5), <perceval.components.unitary_components.Barrier at 0x71aded106d40>),
 ((2, 3),
  <perceval.components.feed_forward_configurator.FFCircuitProvider at 0x71ae702e4340>),
 ((0, 1, 2, 3, 4, 5),
  <perceval.components.unitary_components.Barrier at 0x71aded105cf0>),
 ((0,), <perceval.components.detector.Detector at 0x71aded2cb310>),
 ((1,), <perceval.components.detector.Detector at 0x71aded2ca740>),
 ((2, 3, 4, 5),
  <perceval.components.unitary_components.Barrier at 0x71aded105cc0>),
 ((0, 1),
  <perceval.components.feed_forward_configurator.FFConfigurat

In [9]:
p.components[0]

[[4, 5, 6, 7], <perceval.components.unitary_components.PERM at 0x71ae703c2e90>]

In [11]:
p.components[0][1].U

MatrixN([[0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j],
         [1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
         [0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j]])

In [ ]:
# each component has a matrix, therefore can be easily converted to a ZW circuit